In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Exploração Inicial

In [48]:
df = pd.read_csv("informe_covid_cascavel.csv")
df


/tmp/ipykernel_16798/2584984774.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("informe_covid_cascavel.csv")


,Unnamed: 0,IBGE_RES_PR,IBGE_ATEND_PR,UF_RESIDENCIA,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,EXAME,DATA_DIAGNOSTICO,DATA_CONFIRMACAO_DIVULGACAO,DATA_INICIO_SINTOMAS,ÓBITO COVID-19,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,ORIGEM_NOTIFICACAO
0,8,4104808,4104808,PR,F,77,CASCAVEL,CASCAVEL,188.0,2023-10-12,2023-11-21,2023-10-05,SIM,2023-11-13,2023-11-21,Óbito por COVID-19,NaN,Notifica COVID-19
1,10,4104808,4104808,PR,M,67,CASCAVEL,CASCAVEL,1.0,2023-10-23,2023-11-09,2023-10-23,NÃO,2023-10-25,2023-11-21,Óbito por Outras Causas,NaN,Notifica COVID-19
2,86,4104808,4104808,PR,M,34,CASCAVEL,CASCAVEL,27.0,2023-10-01,2023-11-08,2023-09-18,NÃO,2023-09-29,2023-11-09,Óbito por Outras Causas,NaN,Notifica COVID-19
3,195,4104808,4104808,PR,M,79,CASCAVEL,CASCAVEL,27.0,2023-10-11,2023-10-20,2023-10-07,SIM,2023-10-18,2023-10-20,Óbito por COVID-19,NaN,Notifica COVID-19
4,226,4104808,4104808,PR,M,81,CASCAVEL,CASCAVEL,1.0,2023-10-05,2023-10-17,2023-10-01,SIM,2023-10-15,2023-10-17,Óbito por COVID-19,NaN,Notifica COVID-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87716,2980041,4104808,4104808,PR,M,58,CASCAVEL,CASCAVEL,1.0,2020-03-30,2020-03-31,2020-03-20,NÃO,NaN,NaN,Recuperado,NaN,Notifica COVID-19
87717,2980064,4104808,4104808,PR,M,48,CASCAVEL,CASCAVEL,1.0,2020-04-01,2020-03-28,2020-03-20,NÃO,NaN,NaN,Recuperado,NaN,Notifica COVID-19
87718,2980081,4104808,4104808,PR,M,40,CASCAVEL,CASCAVEL,1.0,2020-03-26,2020-03-27,2020-03-16,NÃO,NaN,NaN,Recuperado,NaN,Notifica COVID-19
87719,2980116,4104808,4104808,PR,F,23,CASCAVEL,CASCAVEL,1.0,2020-03-23,2020-03-24,2020-03-14,NÃO,NaN,NaN,Recuperado,NaN,Notifica COVID-19


In [50]:
df["IDADE_ORIGINAL"].mode()

0    33
Name: IDADE_ORIGINAL, dtype: int64

In [51]:
df.loc[df['ÓBITO COVID-19'] == 'SIM']["IDADE_ORIGINAL"].describe(), df.loc[df['ÓBITO COVID-19'] == 'SIM']["IDADE_ORIGINAL"].mode()


(count    1302.000000
 mean       66.564516
 std        16.917853
 min         0.000000
 25%        56.000000
 50%        69.000000
 75%        79.000000
 max       113.000000
 Name: IDADE_ORIGINAL, dtype: float64,
 0    73
 Name: IDADE_ORIGINAL, dtype: int64)

In [55]:
df["ÓBITO COVID-19"].value_counts(normalize=True)

ÓBITO COVID-19
NÃO    0.985157
SIM    0.014843
Name: proportion, dtype: float64